<a href="https://colab.research.google.com/github/hochschule-pforzheim/project-st23-team-y/blob/main/4_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#KNN Regression

##Load data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Use GPU to work with enhaced power in Colab

import tensorflow as tf

if tf.test.gpu_device_name():
  print(f'Default GPU Device: {tf.test.gpu_device_name()}')
else:
  print("Please install GPU version of TF")

Please install GPU version of TF


In [2]:
#Loading file
df_sent = pd.read_csv('/content/sample_data/df_sent.csv')

In [3]:
df_sent

,Unnamed: 0,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,time_signature_4,time_signature_5,negative,neutral,positive,compound,WordList,polarity_textblob,flair_score,flair_value
0,0,36,295909,0.437,0.607,-5.704,0.0428,0.332000,0.000000,0.1420,...,1,0,0.081,0.722,0.197,0.9778,"['were', 'happens', 'will', 'never', 'this', '...",0.090476,0.999132,POSITIVE
1,1,49,149973,0.578,0.295,-10.379,0.0294,0.792000,0.000000,0.1090,...,1,0,0.053,0.581,0.366,0.9968,"['my', 'must', 'oh', 'might', ""i'll"", 'tell', ...",0.366815,0.995719,POSITIVE
2,2,66,310346,0.423,0.954,-3.157,0.0470,0.002100,0.000000,0.1770,...,1,0,0.046,0.920,0.034,-0.2732,"['a', 'begin', 'is', 'feel', 'will', 'might', ...",0.000000,0.975547,POSITIVE
3,3,57,238373,0.480,0.970,-5.006,0.0595,0.000037,0.309000,0.2900,...,1,0,0.179,0.690,0.131,-0.9489,"['a', 'must', 'oh', 'might', ""i'll"", 'feeling'...",-0.179762,0.998968,NEGATIVE
4,4,13,158773,0.478,0.303,-9.352,0.0275,0.497000,0.000000,0.0855,...,1,0,0.074,0.697,0.229,0.9766,"['my', 'castle', 'mountain', 'a', 'hall', 'giv...",0.300871,0.886031,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,2484,1,284040,0.334,0.814,-14.175,0.0447,0.024700,0.000593,0.5010,...,1,0,0.330,0.564,0.106,-0.9979,"['laughs', 'will', 'turn', 'deceive', 'stone',...",-0.200000,0.732693,NEGATIVE
2485,2485,0,131760,0.620,0.309,-9.209,0.0495,0.788000,0.000000,0.1460,...,1,0,0.066,0.866,0.068,0.3818,"['fire', 'happy', 'meadow', 'a', 'is', 'might'...",0.278636,0.998828,POSITIVE
2486,2486,3,181728,0.686,0.856,-2.717,0.0426,0.089600,0.000000,0.0648,...,1,0,0.043,0.691,0.266,0.9986,"['crazy', 'change', 'for', 'night', 'miss', 'a...",0.180952,0.982428,POSITIVE
2487,2487,29,229896,0.833,0.720,-13.455,0.2180,0.055400,0.000000,0.3390,...,1,0,0.059,0.779,0.162,0.9983,"['happy', 'mention', 'agree', 'will', 'loving'...",0.143328,0.750563,POSITIVE


In [4]:
#remove added index column
df_sent = df_sent.drop('Unnamed: 0', axis = 1)

##Handling categorical data

Now we want to create dummies for our categorical Variables

In [10]:
#encode categorical variable
df_dummies_flair = pd.get_dummies(df_sent.flair_value)

In [11]:
df_dummies_flair

,NEGATIVE,POSITIVE
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
2484,1,0
2485,0,1
2486,0,1
2487,0,1


In [12]:
df_dummies_flair.isna().sum()

NEGATIVE    0
POSITIVE    0
dtype: int64

In [13]:
#save dataset
df_dummies_flair.to_csv('/content/sample_data/df_dummies_flair.csv', index=False)

In [14]:
#load data flair
df_dummies_flair = pd.read_csv('/content/sample_data/df_dummies_flair.csv')

In [15]:
df_dummies_flair

,NEGATIVE,POSITIVE
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
2484,1,0
2485,0,1
2486,0,1
2487,0,1


In [16]:
#check if there are any missing values
#sometimes the saving and reuploading of the dataset led to missing values in the data. That's why we double check
df_dummies_flair.isna().sum()

NEGATIVE    0
POSITIVE    0
dtype: int64

Now we want to create dummies for every relevant word included in the lyrics

In [17]:
#we want to remove stop words from the WordList. For this we modify the used function to create the WordList column

def get_unique_word_list(lyrics):
    word_list = lyrics.split()
    unique_word_list = str(set(word_list)) #when converting the list to a set, the duplicate words are removed. We convert it to a string rather than using a list
    return unique_word_list

In [18]:
#Append a new column "WordList1" which contains a list with all the used columns
df_sent['WordList1'] = df_sent['lyrics'].apply(get_unique_word_list)

We chose to remove the stop words because in the upcoming process we want to know whether any specific words influence the popularity of a song. If the stop words were included in the analysis and they turn out to have an influence on our dependent variable, we wouldn't be able to interprete the results.

In [19]:
#We now want to remove stop words
import spacy
import pandas as pd

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Function to remove stopwords from a text string
def remove_stopwords(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return ' '.join(tokens)

# Apply the function to the 'text_column'
df_sent['WordList1'] = df_sent['WordList1'].apply(remove_stopwords)

In [20]:
df_sent.WordList1

0       { ' hearts ' , ' ' , ' ' , ' ' , ' ' , ' ' , '...
1       { ' ' , ' ' , ' ' , ' yay ' , " " , ' wo ' , '...
2       { ' begin ' , ' disarray ' , ' mask ' , ' eyes...
3       { ' barrier ' , ' eyes ' , ' help ' , ' yeah '...
4       { ' ' , ' meet ' , ' ' , ' woman ' , ' mansion...
                              ...                        
2484    { ' strike ' , ' seize ' , ' lives ' , ' ' , '...
2485    { ' babe ' , ' ' , ' later ' , ' likeand ' , '...
2486    { ' crazy ' , ' ' , ' ' , ' ' , ' change ' , '...
2487    { ' ' , ' ' , ' blind ' , ' yo ' , ' ones ' , ...
2488    { ' ' , ' ' , ' ' , ' eyes ' , ' help ' , ' go...
Name: WordList1, Length: 2489, dtype: object

In [21]:
import re
df_sent['WordList1'] = df_sent['WordList1'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', "", x))

In [22]:
df_sent.WordList1

0         hearts                      greatest    happ...
1                  yay       wo       filled       oh ...
2         begin    disarray    mask    eyes       got ...
3         barrier    eyes    help    yeah    got      ...
4            meet       woman    mansion    dance    k...
                              ...                        
2484      strike    seize    lives       runs       bl...
2485      babe       later    likeand    build    bell...
2486      crazy             change          wish      ...
2487            blind    yo    ones       let       st...
2488               eyes    help    got    red         ...
Name: WordList1, Length: 2489, dtype: object

In [23]:
df_sent['WordList2'] = df_sent['WordList1'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [24]:
df_sent.WordList2

0        hearts greatest happens lost given confess op...
1        yay wo filled oh like tell cry got ta love li...
2        begin disarray mask eyes got come wait like f...
3        barrier eyes help yeah got let feeling lonely...
4        meet woman mansion dance kingdom hall wait he...
                              ...                        
2484     strike seize lives runs blood escape head hey...
2485     babe later likeand build bells ring snow sigh...
2486     crazy change wish space think got ta loved do...
2487     blind yo ones let strong n stepping wish fell...
2488     eyes help got red dreamin like love head gone...
Name: WordList2, Length: 2489, dtype: object

In [25]:
df_sent.WordList3 = df_sent.WordList2.str.split(' ')

<ipython-input-25-942a33587344>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_sent.WordList3 = df_sent.WordList2.str.split(' ')


In [26]:
df_sent['WordList4'] = list(df_sent.WordList3)

In [27]:
df_sent.WordList4

0       [, hearts, greatest, happens, lost, given, con...
1       [, yay, wo, filled, oh, like, tell, cry, got, ...
2       [, begin, disarray, mask, eyes, got, come, wai...
3       [, barrier, eyes, help, yeah, got, let, feelin...
4       [, meet, woman, mansion, dance, kingdom, hall,...
                              ...                        
2484    [, strike, seize, lives, runs, blood, escape, ...
2485    [, babe, later, likeand, build, bells, ring, s...
2486    [, crazy, change, wish, space, think, got, ta,...
2487    [, blind, yo, ones, let, strong, n, stepping, ...
2488    [, eyes, help, got, red, dreamin, like, love, ...
Name: WordList4, Length: 2489, dtype: object

We now want to turn the individual entries into individual binary columns

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df_dummies_words = pd.DataFrame(mlb.fit_transform(df_sent['WordList4']), columns=mlb.classes_, index=df_sent.index)

In [29]:
df_dummies_words.columns

Index(['', '0', '00', '000', '01', '01000111', '013', '015', '018', '02',
       ...
       'zoom', 'zoop', 'zoovier', 'zoovile', 'zoy', 'zq', 'zu', 'zuccini',
       'zucker', 'zyou'],
      dtype='object', length=17215)

For some reason there's a lot of columns starting with numbers. We are guessing, that the index was also dummy encoded

In [30]:
#find all columns with a name that only includes numbers
numerical_columns = df_dummies_words.filter(regex='^\d+$')
print(numerical_columns)

      0  00  000  01  01000111  013  015  018  02  021  ...  965  973  978  \
0     0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
1     0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
2     0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
3     0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
4     0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
...  ..  ..  ...  ..       ...  ...  ...  ...  ..  ...  ...  ...  ...  ...   
2484  0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
2485  0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
2486  0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
2487  0   0    0   0         0    0    0    0   0    0  ...    0    0    0   
2488  0   0    0   0         0    0    0    0   0    0  ...    0    0    0   

      98  982  984  99  991  992  994  
0      0    0    0   0 

In [31]:
df_dummies_words = df_dummies_words.drop(columns=numerical_columns.columns)

In [32]:
df_dummies_words

,,12x,13th,14a,15th,1st,1x,2022spaceman,203x,20k,...,zoom,zoop,zoovier,zoovile,zoy,zq,zu,zuccini,zucker,zyou
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2485,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2486,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2487,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
#we need to remove the first columns, as it is not a word
column_to_drop = 0
df_dummies_words = df_dummies_words.drop(df_dummies_words.columns[column_to_drop], axis=1)

In [34]:
df_dummies_words

,12x,13th,14a,15th,1st,1x,2022spaceman,203x,20k,20th,...,zoom,zoop,zoovier,zoovile,zoy,zq,zu,zuccini,zucker,zyou
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_dummies_words.isna().sum()

12x        0
13th       0
14a        0
15th       0
1st        0
          ..
zq         0
zu         0
zuccini    0
zucker     0
zyou       0
Length: 16758, dtype: int64

In [36]:
#save dataframe with dummy words
df_dummies_words.to_csv('/content/sample_data/df_dummies_words.csv', index=False)


In [37]:
#load word dummies and check whether the saving process was correct
df_dummies_words = pd.read_csv('/content/sample_data/df_dummies_words.csv')

In [38]:
df_dummies_words

,12x,13th,14a,15th,1st,1x,2022spaceman,203x,20k,20th,...,zoom,zoop,zoovier,zoovile,zoy,zq,zu,zuccini,zucker,zyou
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df_dummies_words.isna().sum()

12x        0
13th       0
14a        0
15th       0
1st        0
          ..
zq         0
zu         0
zuccini    0
zucker     0
zyou       0
Length: 16758, dtype: int64

##KNN with chosen variables

###Data preparation

As we have a lot of variables, we figured that it might be better if we concentrate only on important ones for our model. This will also make the interpretation of the results easier. We now want to create a new DataFrame including only relevant variables for our model.

In [40]:
#check which variables are included in the data frame
list(df_sent.columns)

['popularity',
 'duration_ms',
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'lyrics',
 'explicit_False',
 'explicit_True',
 'mode_0',
 'mode_1',
 'key_0',
 'key_1',
 'key_2',
 'key_3',
 'key_4',
 'key_5',
 'key_6',
 'key_7',
 'key_8',
 'key_9',
 'key_10',
 'key_11',
 'track_genre_acoustic',
 'track_genre_afrobeat',
 'track_genre_alt-rock',
 'track_genre_alternative',
 'track_genre_ambient',
 'track_genre_anime',
 'track_genre_black-metal',
 'track_genre_bluegrass',
 'track_genre_blues',
 'track_genre_brazil',
 'track_genre_breakbeat',
 'track_genre_british',
 'track_genre_cantopop',
 'track_genre_chicago-house',
 'track_genre_children',
 'track_genre_chill',
 'track_genre_classical',
 'track_genre_club',
 'track_genre_comedy',
 'track_genre_country',
 'track_genre_dance',
 'track_genre_dancehall',
 'track_genre_death-metal',
 'track_genre_deep-house',
 'track_genre_detroit-techno',
 'track_genre_disco

In [41]:
#find out what the indexes are of specific values in order to drop them using iloc in the upcoming step
print(df_sent.columns.get_loc('key_0'))
print(df_sent.columns.get_loc('positive'))

16
149


In [42]:
df_reg = df_sent.drop(df_sent.columns[16:150], axis=1)
df_reg = df_reg.drop(['lyrics', 'flair_score', 'flair_value', 'WordList'], axis = 1) #drop all string variablen - they will be dummy encoded in the upcoming steps.
#Also remove flair score as this only indicates how high the accuracy of the predicted value is.

In [43]:
df_reg.isna().sum()

popularity           0
duration_ms          0
danceability         0
energy               0
loudness             0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
explicit_False       0
explicit_True        0
mode_0               0
mode_1               0
compound             0
polarity_textblob    0
WordList1            0
WordList2            0
WordList4            0
dtype: int64

In [44]:
df_reg

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,explicit_False,explicit_True,mode_0,mode_1,compound,polarity_textblob,WordList1,WordList2,WordList4
0,36,295909,0.437,0.607,-5.704,0.0428,0.332000,0.000000,0.1420,0.275,176.025,1,0,0,1,0.9778,0.090476,hearts greatest happ...,hearts greatest happens lost given confess op...,"[, hearts, greatest, happens, lost, given, con..."
1,49,149973,0.578,0.295,-10.379,0.0294,0.792000,0.000000,0.1090,0.603,101.171,1,0,0,1,0.9968,0.366815,yay wo filled oh ...,yay wo filled oh like tell cry got ta love li...,"[, yay, wo, filled, oh, like, tell, cry, got, ..."
2,66,310346,0.423,0.954,-3.157,0.0470,0.002100,0.000000,0.1770,0.500,144.553,1,0,1,0,-0.2732,0.000000,begin disarray mask eyes got ...,begin disarray mask eyes got come wait like f...,"[, begin, disarray, mask, eyes, got, come, wai..."
3,57,238373,0.480,0.970,-5.006,0.0595,0.000037,0.309000,0.2900,0.346,136.123,1,0,1,0,-0.9489,-0.179762,barrier eyes help yeah got ...,barrier eyes help yeah got let feeling lonely...,"[, barrier, eyes, help, yeah, got, let, feelin..."
4,13,158773,0.478,0.303,-9.352,0.0275,0.497000,0.000000,0.0855,0.830,141.371,1,0,0,1,0.9766,0.300871,meet woman mansion dance k...,meet woman mansion dance kingdom hall wait he...,"[, meet, woman, mansion, dance, kingdom, hall,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,1,284040,0.334,0.814,-14.175,0.0447,0.024700,0.000593,0.5010,0.676,150.813,1,0,1,0,-0.9979,-0.200000,strike seize lives runs bl...,strike seize lives runs blood escape head hey...,"[, strike, seize, lives, runs, blood, escape, ..."
2485,0,131760,0.620,0.309,-9.209,0.0495,0.788000,0.000000,0.1460,0.664,145.363,1,0,0,1,0.3818,0.278636,babe later likeand build bell...,babe later likeand build bells ring snow sigh...,"[, babe, later, likeand, build, bells, ring, s..."
2486,3,181728,0.686,0.856,-2.717,0.0426,0.089600,0.000000,0.0648,0.733,123.997,1,0,1,0,0.9986,0.180952,crazy change wish ...,crazy change wish space think got ta loved do...,"[, crazy, change, wish, space, think, got, ta,..."
2487,29,229896,0.833,0.720,-13.455,0.2180,0.055400,0.000000,0.3390,0.759,108.141,1,0,0,1,0.9983,0.143328,blind yo ones let st...,blind yo ones let strong n stepping wish fell...,"[, blind, yo, ones, let, strong, n, stepping, ..."


We now need to join the dummy datasets to the new regression dataset

In [45]:
#join the dummies to the dataset
df_reg_final = pd.concat([df_reg, df_dummies_words, df_dummies_flair], axis=1)

In [46]:
df_reg_final

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,zoovier,zoovile,zoy,zq,zu,zuccini,zucker,zyou,NEGATIVE,POSITIVE
0,36,295909,0.437,0.607,-5.704,0.0428,0.332000,0.000000,0.1420,0.275,...,0,0,0,0,0,0,0,0,0,1
1,49,149973,0.578,0.295,-10.379,0.0294,0.792000,0.000000,0.1090,0.603,...,0,0,0,0,0,0,0,0,0,1
2,66,310346,0.423,0.954,-3.157,0.0470,0.002100,0.000000,0.1770,0.500,...,0,0,0,0,0,0,0,0,0,1
3,57,238373,0.480,0.970,-5.006,0.0595,0.000037,0.309000,0.2900,0.346,...,0,0,0,0,0,0,0,0,1,0
4,13,158773,0.478,0.303,-9.352,0.0275,0.497000,0.000000,0.0855,0.830,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,1,284040,0.334,0.814,-14.175,0.0447,0.024700,0.000593,0.5010,0.676,...,0,0,0,0,0,0,0,0,1,0
2485,0,131760,0.620,0.309,-9.209,0.0495,0.788000,0.000000,0.1460,0.664,...,0,0,0,0,0,0,0,0,0,1
2486,3,181728,0.686,0.856,-2.717,0.0426,0.089600,0.000000,0.0648,0.733,...,0,0,0,0,0,0,0,0,0,1
2487,29,229896,0.833,0.720,-13.455,0.2180,0.055400,0.000000,0.3390,0.759,...,0,0,0,0,0,0,0,0,0,1


In [47]:
df_reg_final.isna().sum()

popularity      0
duration_ms     0
danceability    0
energy          0
loudness        0
               ..
zuccini         0
zucker          0
zyou            0
NEGATIVE        0
POSITIVE        0
Length: 16780, dtype: int64

In [48]:
df_reg_final.popularity

,popularity,popularity
0,36,0
1,49,0
2,66,0
3,57,0
4,13,0
...,...,...
2484,1,0
2485,0,0
2486,3,0
2487,29,0


In [49]:
#we realised that now there are two columns named popularity. One is our target column and the other one is the dummy variable for the word popularity.
#we thus need to rename our target column
df_reg_final.columns.values[0] = "target_popularity"

In [50]:
df_reg_final.target_popularity

0       36
1       49
2       66
3       57
4       13
        ..
2484     1
2485     0
2486     3
2487    29
2488    60
Name: target_popularity, Length: 2489, dtype: int64

In [51]:
df_reg_final.isna().sum()

target_popularity    0
duration_ms          0
danceability         0
energy               0
loudness             0
                    ..
zuccini              0
zucker               0
zyou                 0
NEGATIVE             0
POSITIVE             0
Length: 16780, dtype: int64

In [52]:
#Save final dataset before training
df_reg_final.to_csv('/content/sample_data/df_knn.csv', index = False)

In [53]:
#load data anc check whether the saving procedure worked
df_knn = pd.read_csv('/content/sample_data/df_knn.csv')

In [54]:
df_knn

,target_popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,zoovier,zoovile,zoy,zq,zu,zuccini,zucker,zyou,NEGATIVE,POSITIVE
0,36,295909,0.437,0.607,-5.704,0.0428,0.332000,0.000000,0.1420,0.275,...,0,0,0,0,0,0,0,0,0,1
1,49,149973,0.578,0.295,-10.379,0.0294,0.792000,0.000000,0.1090,0.603,...,0,0,0,0,0,0,0,0,0,1
2,66,310346,0.423,0.954,-3.157,0.0470,0.002100,0.000000,0.1770,0.500,...,0,0,0,0,0,0,0,0,0,1
3,57,238373,0.480,0.970,-5.006,0.0595,0.000037,0.309000,0.2900,0.346,...,0,0,0,0,0,0,0,0,1,0
4,13,158773,0.478,0.303,-9.352,0.0275,0.497000,0.000000,0.0855,0.830,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,1,284040,0.334,0.814,-14.175,0.0447,0.024700,0.000593,0.5010,0.676,...,0,0,0,0,0,0,0,0,1,0
2485,0,131760,0.620,0.309,-9.209,0.0495,0.788000,0.000000,0.1460,0.664,...,0,0,0,0,0,0,0,0,0,1
2486,3,181728,0.686,0.856,-2.717,0.0426,0.089600,0.000000,0.0648,0.733,...,0,0,0,0,0,0,0,0,0,1
2487,29,229896,0.833,0.720,-13.455,0.2180,0.055400,0.000000,0.3390,0.759,...,0,0,0,0,0,0,0,0,0,1


In [55]:
df_knn.isna().sum()

target_popularity    0
duration_ms          0
danceability         0
energy               0
loudness             0
                    ..
zuccini              0
zucker               0
zyou                 0
NEGATIVE             0
POSITIVE             0
Length: 16780, dtype: int64